In [1]:
import nglview
import ipywidgets

#pdb_code = "4i23"
pdb_code = "2rgp"
drugbank_code = "DB11963"
ligand_remove = "HYZ"
compare_id = "4hjo"
api_id = "pdbe"
pockets_dir = "pockets"

_ColormakerRegistry()

In [2]:
from biobb_io.api.pdb import pdb

In [3]:
download_pdb = "download.pdb"
prop = {
  "pdb_code": pdb_code,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}

pdb(output_pdb_path=download_pdb,
    properties=prop)

2021-01-26 10:59:03,683 [MainThread  ] [INFO ]  Downloading: 2rgp from: http://www.ebi.ac.uk/pdbe/entry-files/download/pdb2rgp.ent
/anaconda3/envs/biobb_dev_37/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-01-26 10:59:04,080 [MainThread  ] [INFO ]  Writting pdb to: download.pdb
2021-01-26 10:59:04,081 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

In [4]:
view = nglview.show_structure_file(download_pdb, default=True)
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [5]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

In [6]:
pdb_no_ligand = "pdb_no_ligand.pdb"
prop = {
    "ligand": ligand_remove
}

remove_ligand(input_structure_path=download_pdb,
             output_structure_path = pdb_no_ligand,
            properties=prop)

2021-01-26 10:59:10,584 [MainThread  ] [INFO ]  PDB format detected, removing all atoms from residues named HYZ


0

In [7]:
view = nglview.show_structure_file(pdb_no_ligand, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [8]:
from biobb_structure_utils.utils.extract_chain import extract_chain

In [9]:
pdb_single_chain = "extract_chain.pdb"
prop = {
    "chains": ["A"]
}

extract_chain(input_structure_path=pdb_no_ligand,
             output_structure_path = pdb_single_chain,
            properties=prop)

2021-01-26 10:59:15,689 [MainThread  ] [INFO ]  check_structure -i /home/gbayarri_local/projects/BioBB/biobb_vs/biobb_vs/tutorial/ebi_api/pdb_no_ligand.pdb -o extract_chain.pdb --force_save chains --select A

2021-01-26 10:59:15,690 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 10:59:15,691 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure /home/gbayarri_local/projects/BioBB/biobb_vs/biobb_vs/tutorial/ebi_api/pdb_no_ligand.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: None A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  371
 Num. residues with ins. codes:  0
 Num. HETATM residues:  87
 Num. ligands or modified residues:  2
 Num. water mol.:  85
 Num. atoms:  2384
Small mol ligands found
PO4 A81
PO4 A82

Running chains. Options: -

0

In [10]:
view = nglview.show_structure_file(pdb_single_chain, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [11]:
from biobb_io.api.binding_site import binding_site

In [12]:
residues_json = "residues.json"
prop = {
  "pdb_code": pdb_code
}

binding_site(output_json_path=residues_json,
    properties=prop)

2021-01-26 11:01:21,433 [MainThread  ] [INFO ]  Getting binding sites from: https://www.ebi.ac.uk/pdbe/api/pdb/entry/binding_sites/2rgp
2021-01-26 11:01:21,608 [MainThread  ] [INFO ]  3 binding sites found
2021-01-26 11:01:21,609 [MainThread  ] [INFO ]  Writting json to: residues.json


0

In [19]:
import json

with open(residues_json) as json_file: 
    data = json.load(json_file)
    print(json.dumps(data, indent=4))

{
    "2rgp": [
        {
            "details": "BINDING SITE FOR RESIDUE HYZ A 1",
            "evidence_code": "Software",
            "ligand_residues": [
                {
                    "author_insertion_code": null,
                    "author_residue_number": null,
                    "chain_id": null,
                    "chem_comp_id": null,
                    "entity_id": null,
                    "residue_number": null,
                    "struct_asym_id": null
                }
            ],
            "site_id": "AC3",
            "site_residues": [
                {
                    "author_insertion_code": null,
                    "author_residue_number": 856,
                    "chain_id": "A",
                    "chem_comp_id": "PHE",
                    "entity_id": 1,
                    "residue_number": 155,
                    "struct_asym_id": "A",
                    "symmetry_symbol": "1_555"
                },
                {
                

In [42]:
bindingSites = {}
bsites = []
for i, item in enumerate(data[pdb_code]):
    bindingSites[i] = []
    bsites.append(('binding_site' + str(i), i))
    for res in item['site_residues']:
        bindingSites[i].append(res['author_residue_number'])
    print('Residue id\'s for binding site %d: %s' % (i, ', '.join(str(v) for v in bindingSites[i])))
        
mdsel = ipywidgets.Dropdown(
    options=bsites,
    description='Sel. BSite:',
    disabled=False,
)
display(mdsel)

Residue id's for binding site 0: 856, 855, 854, 718, 719, 743, 745, 775, 776, 777, 788, 790, 791, 793, 800, 844
Residue id's for binding site 1: 832, 833, 836, 860, 888
Residue id's for binding site 2: 803, 970, 977, 913, 1093, 1075, 1074


Dropdown(description='Sel. BSite:', options=(('binding_site0', 0), ('binding_site1', 1), ('binding_site2', 2))…

In [43]:
from biobb_vs.utils.box_residues import box_residues

In [51]:
output_box = "box.pdb"
prop = {
    "resid_list": bindingSites[mdsel.value],
    "offset": 6,
    "box_coordinates": True
}

box_residues(input_pdb_path = pdb_single_chain,
            output_pdb_path = output_box,
            properties=prop)

2021-01-26 11:23:30,992 [MainThread  ] [INFO ]  Loading input PDB structure extract_chain.pdb
2021-01-26 11:23:31,045 [MainThread  ] [INFO ]  Mapping residue structure into input structure
2021-01-26 11:23:31,046 [MainThread  ] [INFO ]  Selection of residues successfully matched
2021-01-26 11:23:31,047 [MainThread  ] [INFO ]  Binding site center (Angstroms):     14.494    36.352    93.217
2021-01-26 11:23:31,047 [MainThread  ] [INFO ]  Binding site size (Angstroms):       13.848    14.010    15.127
2021-01-26 11:23:31,048 [MainThread  ] [INFO ]  Volume (cubic Angstroms): 23479
2021-01-26 11:23:31,048 [MainThread  ] [INFO ]  Adding box coordinates
2021-01-26 11:23:31,050 [MainThread  ] [INFO ]  Saving output PDB file (with box setting annotations): box.pdb


0

In [52]:
#view = nglview.show_structure_file(box, default=False)
view2 = nglview.NGLWidget()
#s = view.add_component(pdb_single_chain)
s = view2.add_component(download_pdb)
b = view2.add_component(output_box)

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

# structure
s.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
b.add_representation(repr_type='ball+stick',
                     selection='9999',
                     color='pink', 
                     aspectRatio = 8)
# lines box
b.add_representation(repr_type='distance', 
                     atomPair= atomPair,
                     labelVisible=False,
                     color= 'black')


view2.center()
view2._remote_call('setSize', target='Widget', args=['','600px'])
view2

NGLWidget()

In [ ]:
from biobb_io.api.drugbank import drugbank

In [ ]:
drug = "drug.sdf"
prop = {
  "drugbank_id": drugbank_code
}

drugbank(output_sdf_path=drug,
    properties=prop)

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
ligand = "ligand.pdb"
prop = {
    "input_format": "sdf",
    "output_format": "pdb",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = drug,
            output_path = ligand,
            properties=prop)

In [53]:
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

In [54]:
ligand = "ligand.pdb"
prop = {
    "heteroatoms": [{ "name": ligand_remove }]
}

extract_heteroatoms(input_structure_path = download_pdb,
            output_heteroatom_path = ligand,
            properties=prop)

2021-01-26 11:23:51,679 [MainThread  ] [INFO ]  File ligand.pdb created


0

In [55]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

In [56]:
ligand_H = "ligand_H.pdb"
prop = {
    
}

reduce_add_hydrogens(input_path=ligand,
                    output_path=ligand_H,
                    properties=prop)

2021-01-26 11:23:53,389 [MainThread  ] [INFO ]  Not using any container
2021-01-26 11:23:53,550 [MainThread  ] [INFO ]  reduce -OH -ROTNH3 -ALLALT ligand.pdb > ligand_H.pdb

2021-01-26 11:23:53,553 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 11:23:53,555 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "ligand.pdb"
Database of HETATM connections: "/anaconda3/envs/biobb_dev_37//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
Found 0 hydrogens (0 hets)
Standardized 0 hydrogens (0 hets)
Added 27 hydrogens (27 hets)
Removed 0 hydrogens (0 hets)
If you publish work which uses reduce, please cite:

0

In [57]:
from ipywidgets import HBox

In [58]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(ligand_H)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [59]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [60]:
prep_ligand = "prep_ligand.pdbqt"
prop = {
    "input_format": "pdb",
    "output_format": "pdbqt",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = ligand_H,
            output_path = prep_ligand,
            properties=prop)

2021-01-26 11:23:57,081 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2021-01-26 11:23:57,083 [MainThread  ] [INFO ]  Not using any container
2021-01-26 11:23:57,227 [MainThread  ] [INFO ]  /anaconda3/bin/obabel -ipdb ligand_H.pdb -opdbqt -Oprep_ligand.pdbqt  

2021-01-26 11:23:57,229 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 11:23:57,229 [MainThread  ] [INFO ]  1 molecule converted



0

In [61]:
from biobb_structure_utils.utils.str_check_add_hydrogens import str_check_add_hydrogens

In [62]:
prep_receptor = "prep_receptor.pdbqt"
prop = {
    "charges": True,
    "mode": "auto"
}

str_check_add_hydrogens(input_structure_path = pdb_single_chain,
            output_structure_path = prep_receptor,
            properties=prop)

2021-01-26 11:23:59,507 [MainThread  ] [INFO ]  check_structure -i /home/gbayarri_local/projects/BioBB/biobb_vs/biobb_vs/tutorial/ebi_api/extract_chain.pdb -o prep_receptor.pdbqt --force_save add_hydrogen --add_charges --add_mode auto

2021-01-26 11:23:59,508 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 11:23:59,509 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.7.2                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-20                  =

Structure /home/gbayarri_local/projects/BioBB/biobb_vs/biobb_vs/tutorial/ebi_api/extract_chain.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution: None A

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  371
 Num. residues with ins. codes:  0
 Num. HETATM residues:  87
 Num. ligands or modified residues:  2
 Num. water mol.:  85
 Num. atoms:  2384
Small mol ligands found
PO4 A81
PO4 A82


0

In [64]:
from biobb_vs.vina.autodock_vina import autodock_vina

In [65]:
output_vina_pdbqt = "output_vina.pdbqt"
output_vina_log = "output_vina.log"

autodock_vina(input_ligand_pdbqt_path = prep_ligand,
             input_receptor_pdbqt_path = prep_receptor,
             input_box_path = output_box,
            output_pdbqt_path = output_vina_pdbqt,
             output_log_path = output_vina_log)

2021-01-26 11:24:02,988 [MainThread  ] [INFO ]  prep_receptor.pdbqt file ends with END, cleaning
2021-01-26 11:24:02,992 [MainThread  ] [INFO ]  Executing AutoDock Vina
2021-01-26 11:24:02,992 [MainThread  ] [INFO ]  Not using any container
2021-01-26 11:24:55,141 [MainThread  ] [INFO ]  vina --ligand prep_ligand.pdbqt --receptor prep_receptor.pdbqt --center_x=14.494 --center_y=36.352 --center_z=93.217 --size_x=13.848 --size_y=14.010 --size_z=15.127 --out output_vina.pdbqt --log output_vina.log

2021-01-26 11:24:55,142 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 11:24:55,142 [MainThread  ] [INFO ]  #################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and   

0

In [66]:
models = 'all'
#models = '/0 or /1 or /4'

v0 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [67]:
from Bio.PDB import PDBParser
parser = PDBParser(QUIET = True)
structure = parser.get_structure("protein", output_vina_pdbqt)
models = []
for i, m in enumerate(structure):
    models.append(('model' + str(i), i))
    
mdsel = ipywidgets.Dropdown(
    options=models,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

Dropdown(description='Sel. model:', options=(('model0', 0), ('model1', 1), ('model2', 2), ('model3', 3)), valu…

In [77]:
from biobb_vs.utils.extract_model_pdbqt import extract_model_pdbqt

In [78]:
output_pdbqt_model = "output_model.pdbqt"
prop = {
    "model": mdsel.value + 1
}

extract_model_pdbqt(input_pdbqt_path = output_vina_pdbqt,
             output_pdbqt_path = output_pdbqt_model,
            properties=prop)

2021-01-26 11:25:40,167 [MainThread  ] [INFO ]  Saving model 2 to output_model.pdbqt


0

In [79]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(output_pdbqt_model)

v0._set_size('500px', '')
v1._set_size('500px', '')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [80]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [81]:
output_pdb_model = "output_model.pdb"
prop = {
    "input_format": "pdbqt",
    "output_format": "pdb",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = output_pdbqt_model,
             output_path = output_pdb_model,
            properties=prop)

2021-01-26 11:25:43,767 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2021-01-26 11:25:43,769 [MainThread  ] [INFO ]  Not using any container
2021-01-26 11:25:43,813 [MainThread  ] [INFO ]  /anaconda3/bin/obabel -ipdbqt output_model.pdbqt -opdb -Ooutput_model.pdb  

2021-01-26 11:25:43,815 [MainThread  ] [INFO ]  Exit code 0

2021-01-26 11:25:43,816 [MainThread  ] [INFO ]  1 molecule converted



0

In [82]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

In [83]:
output_structure = "output_structure.pdb"

cat_pdb(input_structure1 = pdb_single_chain,
             input_structure2 = output_pdb_model,
             output_structure_path = output_structure)

2021-01-26 11:25:44,935 [MainThread  ] [INFO ]  File output_structure.pdb created


0

In [84]:
# download 4hjo for comparing purposes
compare_structure = compare_id + ".pdb"
prop = {
  "pdb_code": compare_id,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}
compare_structure
pdb(output_pdb_path=compare_structure,
    properties=prop)

2021-01-26 11:25:45,744 [MainThread  ] [INFO ]  Downloading: 4hjo from: http://www.ebi.ac.uk/pdbe/entry-files/download/pdb4hjo.ent
/anaconda3/envs/biobb_dev_37/lib/python3.7/site-packages/urllib3/connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-01-26 11:25:46,081 [MainThread  ] [INFO ]  Writting pdb to: 4hjo.pdb
2021-01-26 11:25:46,083 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

In [85]:
# load reference and output
view = nglview.NGLWidget()
#v1 = view.add_component(compare_structure)
v1 = view.add_component(download_pdb)
v2 = view.add_component(output_structure)
v1.clear()
v1.add_representation(repr_type='licorice', 
                        selection='HYZ',
                      #selection='AQ4',
                     radius=0.5)

view._remote_call('setSize', target='Widget', args=['','600px'])
view

something = 'kk'

# align reference and output
code = """
console.log(%s)
var stage = this.stage;
var clist_len = stage.compList.length;
var i = 0;
var s = [];
//console.log(clist_len)
//console.log(stage.compList[0])
//console.log(stage.compList)
for(i = 0; i <= clist_len; i++){
    //console.log(stage.compList[i])
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined) {        
       s.push(stage.compList[i])
    }
}

NGL.superpose(s[0].structure, s[1].structure, true)
s[ 0 ].updateRepresentations({ position: true })
s[ 0 ].autoView()
""" % something

view._execute_js_code(code)
view

NGLWidget()

In [ ]:
#v0 = nglview.show_structure_file(compare_structure)
v0 = nglview.show_structure_file(download_pdb)

v1 = nglview.show_structure_file(output_structure)

v0._set_size('500px', '500px')
v1._set_size('500px', '500px')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])